# 벡터 DB : chroma vs pinecorn
- Chroma : 인메모리 DB, 로컬메모리 DB
- Pinecone : 클라우드 vecter DB
- pip install pinecone-client langchain-pinecone
    (pinecone console에서 api key 생성 -> .env (PINECONE_API_KEY))

# 1. Knowledge Base 구성을 위한 데이터 생성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader("./tax_docs/소득세법(법률)(제20615호)(20250701).docx")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
documents_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
len(documents_list)

In [2]:
# embedding : upstage API text-embedding-3-large
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
    #model="embedding-query"
)

In [ ]:
# pinecone vecter database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
# 데이터를 처음 업로드할 때
index_name = 'tax-index-upstage'
# database = PineconeVectorStore.from_documents(
#     documents=documents_list,
#     embedding=embedding,
#     index_name=index_name,
# )
# 업로드한 벡터DB 가져올 때
database = PineconeVectorStore(
    embedding=embedding,  # 질문을 임베딩하여 유사도 검색
    index_name=index_name,
)

c:\Users\Admin\anaconda3\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 제공되는 prompt를 활용하여 답변 생성

In [4]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [5]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [6]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke({'query':query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 정확한 계산에 따라 다르지만, 일반적으로 근로소득공제와 각종 공제 후 과세표준에 따라 결정됩니다. 참고로, 2014년 기준으로 총급여액이 5천만원인 경우, 근로소득공제는 최대 2천만원으로 제한됩니다. 따라서 과세표준은 약 3천만원 정도이며, 이에 따라 소득세는 대략적으로 계산할 수 있습니다.'}